# Kencan AI Assistant - Google Colab Deployment

This notebook sets up Kencan on Google Colab with free GPU access.

**Setup Steps:**
1. Enable GPU: Runtime → Change runtime type → GPU
2. Run all cells in order
3. Copy the ngrok URL and configure your local agent
4. Start making requests!

In [ ]:
# Install dependencies
!pip install -q transformers torch accelerate flask flask-cors pyngrok requests

In [ ]:
# Import libraries
import os
import json
from flask import Flask, request, jsonify
from pyngrok import ngrok
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from threading import Thread

print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")

In [ ]:
# Configuration
MODEL_NAME = "microsoft/phi-2"  # Free, fast model that works on Colab
NGROK_AUTH_TOKEN = "YOUR_NGROK_TOKEN"  # Get free token from ngrok.com

# Set your ngrok token
if NGROK_AUTH_TOKEN != "YOUR_NGROK_TOKEN":
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
# Load model
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print("Model loaded successfully!")

In [ ]:
# Create Flask API with command queue and conversation memory
import uuid
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

# Command queue for pending commands
command_queue = []
command_results = {}

# Conversation memory (last N exchanges)
conversation_history = []
MAX_HISTORY = 10

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "healthy", "gpu": torch.cuda.is_available()})

@app.route('/command', methods=['POST'])
def process_command():
    try:
        data = request.json
        if not data:
            return jsonify({'success': False, 'error': 'No JSON data provided'}), 400
        
        user_input = data.get('input', '')
        if not user_input:
            return jsonify({'success': False, 'error': 'No input provided'}), 400
        
        # Build context from conversation history
        history_context = ''
        if conversation_history:
            history_context = 'Previous conversation:\n'
            for h in conversation_history[-MAX_HISTORY:]:
                history_context += f"User: {h['user']}\nAssistant: {h['assistant']}\n"
            history_context += '\n'
        
        # Generate response with context
        prompt = f"""You are Kencan, an AI assistant that helps control a Windows PC.
Available actions: open_browser, new_tab, close_tab, search_web, click_element, type_text,
install_program, uninstall_program, run_command, open_application, create_file, read_file, delete_file, research.

{history_context}Current request: {user_input}

Respond with a JSON object containing 'action' and 'parameters' fields.
Example: {{"action": "search_web", "parameters": {{"query": "weather today"}}}}
Response:"""
        
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract just the response part (after the prompt)
        response = response.split('Response:')[-1].strip()
        
        # Save to conversation history
        conversation_history.append({'user': user_input, 'assistant': response})
        if len(conversation_history) > MAX_HISTORY * 2:
            conversation_history.pop(0)
        
        # Create command with ID and add to queue
        command_id = str(uuid.uuid4())
        try:
            # Try to parse as JSON command
            import re
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                command_json = json.loads(json_match.group())
                command_json['id'] = command_id
                command_queue.append(command_json)
        except json.JSONDecodeError:
            pass  # Response wasn't valid JSON, that's ok
        
        return jsonify({
            'success': True,
            'response': response,
            'command_id': command_id,
            'model': MODEL_NAME
        })
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/commands/pending', methods=['GET'])
def get_pending_commands():
    """Return pending commands and clear the queue"""
    global command_queue
    commands = command_queue.copy()
    command_queue = []
    return jsonify({'commands': commands})

@app.route('/commands/<command_id>/result', methods=['POST'])
def receive_command_result(command_id):
    """Receive execution result from local agent"""
    try:
        data = request.json
        command_results[command_id] = data
        return jsonify({'success': True, 'message': 'Result received'})
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/conversation/clear', methods=['POST'])
def clear_conversation():
    """Clear conversation history"""
    global conversation_history
    conversation_history = []
    return jsonify({'success': True, 'message': 'Conversation cleared'})

@app.route('/conversation/history', methods=['GET'])
def get_conversation_history():
    """Get conversation history"""
    return jsonify({'history': conversation_history})

@app.route('/finetune', methods=['POST'])
def finetune():
    return jsonify({'message': 'Fine-tuning endpoint - implement with your data'})

print('Flask app created with command queue and conversation memory!')

In [ ]:
# Start server with ngrok
port = 5000

# Start ngrok tunnel
public_url = ngrok.connect(port)
print(f"\n{'='*60}")
print(f"🚀 Kencan AI Assistant is running!")
print(f"📡 Public URL: {public_url}")
print(f"{'='*60}\n")
print("Copy the URL above and configure it in your local agent's config/settings.json")
print("\nEndpoints:")
print(f"  - Health check: {public_url}/health")
print(f"  - Send command: {public_url}/command")
print(f"  - Pending commands: {public_url}/commands/pending")
print(f"  - Command result: {public_url}/commands/<id>/result")
print(f"  - Conversation history: {public_url}/conversation/history")
print(f"  - Clear conversation: {public_url}/conversation/clear")
print(f"  - Fine-tune: {public_url}/finetune")

# Run Flask app
from flask import Flask
from werkzeug.serving import run_simple

run_simple('0.0.0.0', port, app, use_reloader=False, use_debugger=False)